# Imports

In [ ]:
import torch
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [ ]:
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
from src.data import movielens_dataloaders
from src.hyperparams_tuning import tune_params, final_train

# MovieLens 1M

In [ ]:
max_epochs_ml = 10

## HyperParameter Tuning

In [ ]:
study_ml, df_tuning_results_ml = tune_params(
    model_name='VAE', 
    dataset_name='movielens', 
    max_epochs=max_epochs_ml,
    n_trials=50, 
    device=device
    )

In [ ]:
display(df_tuning_results_ml.sort_values(by='value').head(15))
best_params_ml = study_ml.best_params

## Full Training
- Use the best hyperparameters to build the final model
- Train the final model on the train+validation data sets (full_train)
- Test it against the test set for final results

In [ ]:
test_loss_ml, final_model_ml, test_mrr_ml = final_train(
    model_name='VAE', 
    dataset_name='movielens', 
    max_epochs=max_epochs_ml,
    params=best_params_ml, 
    device=device
    )
print(f'Final test loss = {test_loss_ml}\nFinal test MRR = {test_mrr_ml}')

# Netflix

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=.